# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import pickle

In [41]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [6]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [49]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[914:1171]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=99.0.4844.74)
Stacktrace:
Backtrace:
	Ordinal0 [0x0112CF43+2608963]
	Ordinal0 [0x010BE8C1+2156737]
	Ordinal0 [0x00FB4190+1065360]
	Ordinal0 [0x00FA66A3+1009315]
	Ordinal0 [0x00FA638F+1008527]
	Ordinal0 [0x00FA59A0+1005984]
	Ordinal0 [0x00FA48D0+1001680]
	Ordinal0 [0x00FA4ED8+1003224]
	Ordinal0 [0x00FA4E68+1003112]
	Ordinal0 [0x00FAAB55+1026901]
	Ordinal0 [0x00FAAD96+1027478]
	Ordinal0 [0x00FA608D+1007757]
	Ordinal0 [0x00FA65A5+1009061]
	Ordinal0 [0x00FA638F+1008527]
	Ordinal0 [0x00FA59A0+1005984]
	Ordinal0 [0x00FA48D0+1001680]
	Ordinal0 [0x00FA4ED8+1003224]
	Ordinal0 [0x00FA4E68+1003112]
	Ordinal0 [0x00FAAFF0+1028080]
	Ordinal0 [0x00FA608D+1007757]
	Ordinal0 [0x00FA65A5+1009061]
	Ordinal0 [0x00FA638F+1008527]
	Ordinal0 [0x00FA59A0+1005984]
	Ordinal0 [0x00FA48D0+1001680]
	Ordinal0 [0x00FA4ED8+1003224]
	Ordinal0 [0x

In [50]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text


In [48]:
#user.to_pickle('user_1987-1023.pkl', protocol = 4)

In [32]:
209+ 129

338

In [47]:
for i in range(2022,2027):
    if isbn13[i] == '9788959139217':
        print(i)

2023


In [22]:
user1 = pd.read_pickle('user_4209-4338.pkl')
user2 = pd.read_pickle('user_4339-4473.pkl')

In [23]:
pd.concat([user1, user2, user3])

,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,[9.3],"[67%, 33%, 0%, 0%, 0%]","[공고한 몽매함을 돌파하는 시선, 예찬 - 미셸 투르니에, 유머와 위트로 담아낸 삶...","[필*아, B******k, 은**니, 낙*, t*****a, A***N, 배*훈,...","[5점, 5점, 5점, 5점, 5점, 5점, 4점, 4점, 4점]","[\n이 에세이(Esse)에는 아홉 개의 챕터, 82편의 글들로 구성되어 있는데, ..."
1,[],[],[],[],[],[]
2,[9.1],"[64%, 32%, 5%, 0%, 0%]","[하루가 어찌 흐르는지, 그럴 땐 바로 토끼시죠 / 지수 / 카멜북스, 그럴땐 바로...","[꼼*, 글***재, 행***활, 져*, t******5, 만*, 미*, 연*지, ...","[4점, 5점, 5점, 5점, 3점, 5점, 5점, 5점, 5점, 5점]",[\n블로그에 글을 쓰는 건 정말 오랜만이다. 그동안 나는 책을 읽고 리뷰를 쓰겠다...
3,[9.4],"[67%, 33%, 0%, 0%, 0%]","[배우를 찾습니다, 배우를 찾습니다, 배우를 꿈꾸는 나의 오랜 친구에게 주는 선물.]","[j********7, k*****6, s******1]","[5점, 5점, 4점]",[\n배우 지망생이라면 Must-Buy(반드시 사야할) 책입니다.\n \n이 책은 ...
4,[8.9],"[62%, 31%, 6%, 0%, 0%]","[직장에서 꼰대는 안 되어야지.(파블 15기 11-6), '나님'을 지키는 나만의 ...","[샨**티, 키*스, J*y, 설*, 미**빈, 쭈**미, 노***리, s*****...","[5점, 4점, 4점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n 직장 생활을 오래 하다보면 알게 모르게 꼰대가 되어가는 것은 아닌지 반문하...
...,...,...,...,...,...,...
157,[9.5],"[83%, 17%, 0%, 0%, 0%]","[드라이빙 미스 노마, Book_흐름출판>드라이빙 미스 노마, 여행하는 할머니를 통...","[장**장, 올*브, 늘**럼, y*b, 퀸**림, 해*름, k*******2, 구...","[5점, 4점, 4점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]","[\n드라이빙 미스 노마내가 고등학교 3학년이 되던 1988년 1월,사랑하는 엄마가..."
158,[],[],[],[],[],[]
159,[9.3],"[76%, 24%, 0%, 0%, 0%]","[안타까운 '패자'의 역사. '황하에서 천산까지', 동아시아 소수민족의 역사를 보다...","[백**생, a***6, a******x, q******n, 나**을, 제*스, a...","[5점, 5점, 5점, 5점, 5점, 5점, 4점, 4점, 5점, 5점]","[\n(몇 년 전의 독후감을 써 온 것인데, 정말 내가 이렇게 썼던가??) \n\n..."
160,[],[],[],[],[],[]
